## Este Notebook tem como objetivo:
* ### extrair o texto das páginas de cada semente, de 10 sites. São eles:
  1. Syngenta
  2. NK sementes (também são da Syngenta)
  3. TMG
  4. Don Mario cerrado
  5. Don Mario Sul
  6. Brasmax Cerrado
  7. Brasmax Sul
  8. Nidera
  9. Brevant
  10. KWS

In [1]:
url_1 = 'https://www.portalsyngenta.com.br/sementes/syngenta-soja'
url_2 = 'https://www.portalsyngenta.com.br/sementes/nk-soja'
url_3 = 'https://www.tmg.agr.br/ptbr/cultivares/soja'
url_4 = 'https://www.donmario.com/pt-br/catalogo-cerrado/'
url_5 = 'https://www.donmario.com/pt-br/catalogo-sul/'
url_6 = 'https://www.brasmaxgenetica.com.br/cultivar-regiao-cerrado/'
url_7 = 'https://www.brasmaxgenetica.com.br/cultivar-regiao-sul/'
url_8 = 'https://www.niderasementes.com.br/soja/'
url_9 = 'https://www.brevant.com.br/produtos/soja.html'
url_10 = 'https://www.kws.com/br/pt/produtos/soja/portfolio-completo-soja/'

In [2]:
from html2text import html2text
from urllib.request import urlopen
from urllib.parse import urljoin
from bs4 import BeautifulSoup

# 1. Syngenta
* Analisando o html da página, vemos que todos os cards das sementes tem:
  * < div class = "card-text-2 card-portfolio"
  
* Para obter os links específicos de cada semente, temos que usar `urljoin`
  * link recolhido no html + link principal

* Entretanto, alguns links parciais estão incompletos.
* Para corrigir o erro, temos que usar um if dentro da def `scrap_and_colect_syngenta` para adicionar a parte que falta _(sementes/)_, caso não tenha.

In [3]:
def scrap_and_colect_syngenta(url, class_card):
    
    """Abre o link principal das sementes de soja da Syngenta e da NK sementes.
    Recebe:
    * <url> a ser scrapped; 
    * <class_card> que contém os links das sementes.
    ---> Retorna uma lista com os links de cada semente"""
    
    html = urlopen(url)
    scrap = BeautifulSoup(html, 'html.parser')

    cards = scrap.find_all('div', {'class': class_card})
    
    links = []

    for link in cards:
        link = link.a['href']
        if 'sementes' in link:
            links.append(urljoin('https://www.portalsyngenta.com.br', link))
        else: 
            link = 'sementes/'+link
            links.append(urljoin('https://www.portalsyngenta.com.br', link))
    
    return links                   

In [4]:
links_1 = scrap_and_colect_syngenta(url_1, "card-text-2 card-portfolio") 
links_1    # link para cada semente de soja Syn 

['https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn15600-ipro',
 'https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn16861-ipro',
 'https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn1059-rr',
 'https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn1163-rr',
 'https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn1258-rr',
 'https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn1281-rr',
 'https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn1683-ipro',
 'https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn1785-ipro',
 'https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn1687-ipro',
 'https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn13610-ipro',
 'https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn15640-ipro',
 'https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn15630-ipro',
 'https://www.portalsyngenta.com.br/sementes/syngenta-soja/syn1562-ipro',
 'https://www.portalsyngenta.com.br/semen

# 2. NK sementes (Syngenta)
* Analisando o html da página, vemos que todos os cards das sementes tem:
  * < div class = "card-text-2 card-portfolio"

In [5]:
links_2 = scrap_and_colect_syngenta(url_2, "card-text-2 card-portfolio") 
links_2    # link para cada semente de soja NK

['https://www.portalsyngenta.com.br/sementes/nk-soja/nk-8770-ipro',
 'https://www.portalsyngenta.com.br/sementes/nk-soja/nk-7777-ipro',
 'https://www.portalsyngenta.com.br/sementes/nk-soja/nk-8448-ipro',
 'https://www.portalsyngenta.com.br/sementes/nk-soja/nk-8301-ipro',
 'https://www.portalsyngenta.com.br/sementes/nk-soja/nk-7201-ipro',
 'https://www.portalsyngenta.com.br/sementes/nk-soja/nk-6201-ipro']

# 3. TMG
* Analisando o html da página, vemos que s cards das sementes tem essa div, porém com CSS **FLEX container**:
  * < div class = "cultivar-link-v" flex
* No caso da TMG, os links não são parciais, já vem completos.
* Por isso a função pode ser diferente, sem o urljoin.
* Entretanto, com a página em .js dinâmica, temos que usar `selenium` e `chromedriver`
  * REF: https://stackoverflow.com/questions/62165635/how-to-scrape-data-from-flexbox-element-container-with-python-and-beautiful-soup
  * download Chromedrive: https://chromedriver.storage.googleapis.com/index.html?path=90.0.4430.24/
    * O caminho do driver dentro do pc será usado. 
    * Exemplo: /home/anandaheino/chromedriver
    
## * Tentativa de função: 
OBS: erro de não fechar o browser!!!

In [57]:
# !pip install selenium

from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from time import sleep

def scrap_and_colect_css(url, class_card):
    
    """Abre o link principal das sementes de soja.
    Recebe:
    * <url> a ser scrapped; 
    * <class_card> que contém os links das sementes.
    ---> Retorna uma lista com os links de cada semente"""

    
    chrome_options = Options()
    
    chrome_options.add_argument('disable-notifications')
    chrome_options.add_argument('--disable-infobars')
    chrome_options.add_argument("disable-infobars")
    chrome_options.add_experimental_option("prefs", 
                                           {"profile.default_content_setting_values.notifications": 2})
    
    browser = webdriver.Chrome(executable_path='/home/anandaheino/chromedriver', options = chrome_options) 
    browser.get(url)
   
    delay = 5 #secods

    #try:
    WebDriverWait(browser, delay)
    print ("\nPage is ready")
    sleep(5)
    browser.minimize_window()
    #html = browser.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
    html = browser
    #print(html)
    scrap = BeautifulSoup(html, "html.parser")
    cards = scrap.find_all('div', {'class': class_card})

    links = list(link.a['href'] for link in cards)
    return links

    browser.close() 
    browser.quit()
    
    #except TimeoutException:
        #print ("Loading took too much time! -Try again")

    #finally:
        #browser.quit()
        
    

In [ ]:
# FUNÇÃO COM ERRO, NÃO FECHA O DRIVER APÓS SCRAP A PÁGINA!!!
# PEGUEI OS LINKS QUE APARECERAM COMO RETORNO.
# NÃO SALVOU NA LISTA links_3.

    #cod:
# links_3 = scrap_and_colect(url_3, "cultivar-link-v") 
# links_3    # link para cada semente de soja da TMG

In [61]:
links_3_tmg = ['https://www.tmg.agr.br/ptbr/cultivar/tmg-2375-ipro', 
'https://www.tmg.agr.br/ptbr/cultivar/tmg-2379-ipro',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-7058-ipro',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-2165-ipro',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-4377',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-7368-ipro',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-2757-ipro',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-2776-ipro',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-2374-ipro',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-2359-ipro',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-2285-ipro',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-1180-rr',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-7063-ipro',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-7062-ipro',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-7067-ipro',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-4182',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-7262-rr',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-2185-ipro',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-7061-ipro',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-2381-ipro',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-2383-ipro',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-1288-rr',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-2181-ipro',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-1188-rr',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-2378-ipro',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-2173-ipro',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-4185',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-7363-rr',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-7260-ipro',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-133-rr',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-132-rr',
'https://www.tmg.agr.br/ptbr/cultivar/anta-82',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-1182-rr',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-2182-ipro',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-7161-rr',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-2183-ipro',
'https://www.tmg.agr.br/ptbr/cultivar/tmg-2286-ipro']